<a href="https://colab.research.google.com/github/folkn/Numerical-Methods/blob/master/Linear_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLVING LINEAR SYSTEMS in PYTHON
**Warapong Narongrit and Techin Choklertwatana**

references
https://numericalmethodsece101.weebly.com/doolittlersquos-method.html
https://numericmethod.weebly.com/uploads/2/5/0/8/25086597/lkm-8-e.pdf

We import the numPy library with the shortcut 'np'

In [0]:
import numpy as np
from scipy import linalg


We are interested in solving simultaneous linear equations.
FIRST, we input the simultaneous equation in matrix form $AX=B$ :

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13} & \dots & A_{1n} \\
    A_{21}       & A_{22} & A_{23} & \dots & A_{2n} \\
    A_{d1}       & A_{d2} & A_{d3} & \dots & A_{dn}
\end{bmatrix} 
\begin{bmatrix}
     X_1 \\
     \dots \\
     X_n 
\end{bmatrix}
= \begin{bmatrix}
    B_1 \\
    \dots \\
    B_n
\end{bmatrix}
$$


In [19]:
#INPUT MATRIX HERE#
A = np.array([[2., 1.], [5.,7.]])
B = np.array([[11.], [13.]])

################################################################
print('Input:\nA=\n' + str(A)); print('B=\n' + str(B))   #Prints matrices
print('\nOutput:\neigenvalues= ' + str(np.linalg.eigvals(A))) #Checks eigenvalues
print ('\nActual Solution: X=\n' + str(np.linalg.solve(A,B))) #Exact solution as calculated by numpy

Input:
A=
[[2. 1.]
 [5. 7.]]
B=
[[11.]
 [13.]]

Output:
eigenvalues= [1.14589803 7.85410197]

Actual Solution: X=
[[ 7.11111111]
 [-3.22222222]]


## Analytical Solution using Gauss Elimination

## Analytical Solution using LU Factorization 

This method uses the idea that all matrices (except those with some diagonal elements being zero) can be factorized into products Lower and upper triangular matrices:

$$A = LU$$ 

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13}  \\
    A_{21}       & A_{22} & A_{23}  \\
    A_{31}       & A_{32} & A_{33} 
\end{bmatrix} =
\begin{bmatrix}
    l_{11}       & 0 & 0  \\
    l_{21}       & l_{22} & 0  \\
    l_{31}       & l_{32} & l_{33} 
\end{bmatrix} 
\begin{bmatrix}
    u_{11}       & u_{12} & u_{13}  \\
    0       & u_{22} & u_{23}  \\
    0       & 0 & 0
\end{bmatrix}
$$

There is more than one way of factorizing, so three **LU Factorization** methods are used:
1. **Doolittle** - all Diagonals of $L$ are $1$    ($l_{11} = l_{22} = l_{nn} = 1$)
2. **Crout** - all Diagonals of $U$ are $1$     ($u_{11} = u_{22} = u_{nn} = 1$)
3. **Cholesky** - $U = L^\text{T}$ or equivalently $L=U^\text{T}$ such that $A=LL^\text{T}$ (*works only if A=positive definite(**positive eigenvalues**) and symmetric **($A=A^\text{T}$)**
---
**The procedure to solving linear systems is:**
1. LU-Factorize the coefficients matrix $A$ using one of the methods
2. Let $LY = B$ where $Y$ is an intermediate variable matrix. Solve for $Y$
3. Let $UX = Y$ then solve for the unknowns $X$ 

I first create functions for steps 2 and 3 since all methods will require these steps

In [0]:
def LY_B (L, B) : #Step 2
    Y =  np.matmul((np.linalg.inv(L)), B)
    return Y

def UX_Y (U, Y) : #Step 3
    X = np.matmul(np.linalg.inv(U) , Y)
    return X

Now I will perform the three methods of factorization methods

### 1. Doolittle's Method https://www.codewithc.com/lu-decomposition-algorithm-flowchart/ https://vismor.com/documents/network_analysis/matrix_algorithms/S4.SS2.php ###

In the code, we convert the $LU$ vector multiplication into a dot multiplcation problem for better scaling.

In [18]:
#1. Doolittle Factorization https://stackoverflow.com/questions/48715594/why-is-my-profs-version-of-lu-decomposition-faster-than-mine-python-numpy
def doolittle(A) : 
    N = np.size(A, 0)    # Number of Rows (Equations)
    
    # Create empty L U matrices with the dimensions of A
    L = np.zeros_like(A)  
    U = np.zeros_like(A)


    for k in range(N):   #For each equation (row) k
        #Determining Diagonal Elements
        L[k, k] = 1      #Set the L diagonal to 1  (Doolittle Requirement)
        U[k, k] = (A[k, k] - np.dot(L[k, :k], U[:k, k])) / L[k, k] 
        #Determining Other Elements
        for j in range(k+1, N):
            U[k, j] = (A[k, j] - np.dot(L[k, :k], U[:k, j])) / L[k, k]
        for i in range(k+1, N):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], U[:k, k])) / U[k, k]
    return L, U
    
L, U = doolittle(A);
print ('L=\n' +str(L)); print('U=\n' + str(U))

#Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')


L=
[[1.  0. ]
 [2.5 1. ]]
U=
[[2.  1. ]
 [0.  4.5]]
LY=B
Y=
[[ 11. ]
 [-14.5]]
UX=Y
X=
[[ 7.11111111]
 [-3.22222222]]	ANS


In [13]:
#Doolittle using SciPy's functinos
def scipy_plu(A) :
    L, U = linalg.lu(A, True)
    return L,U
L,U = scipy_plu(A)
print ('L=\n' +str(L)); print('U=\n' + str(U))

 #Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')

L=
[[0.4 1. ]
 [1.  0. ]]
U=
[[ 5.   7. ]
 [ 0.  -1.8]]


NameError: ignored

### 2. Crout's Method

In [0]:
#2. Crout Factorization 


### 3. Cholesky's Method

In [20]:
#3. Cholesky's Method
def cholesky(A) :
    L = np.linalg.cholesky(A) #wow!
    U = L.T #Cholesky requirement U=transpose(L)
    return L,U

#Try the cholesky, and catch any errors
try:
    L,U = cholesky(A)
except Exception as err:
    print (err)
print ('L=\n' +str(L)); print('U=\n' + str(U))
#Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')

L=
[[2.23606798 0.        ]
 [1.34164079 0.4472136 ]]
U=
[[2.23606798 1.34164079]
 [0.         0.4472136 ]]
LY=B
Y=
[[0.4472136 ]
 [3.13049517]]
UX=Y
X=
[[-4.]
 [ 7.]]	ANS


## Numerical Solutions using Iterations

This method uses repeated calculations to **approximate** the solutions.

## 1. Jacobi Iteration
Assumes that the diagnoal elements are non-zero https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf https://www.kth.se/social/files/5885d039f2765429974418ce/Lab1.pdf
https://gist.github.com/angellicacardozo/3a0891adfa38e2c4187612e57bf271d1

In [29]:
###Please Input 'Guess' Here as a column vector###
X = np.array([[2.9], [-3.5]])
Iterations = 1000
Tolerance = 0.01 #Absolute Tolerance           Stop calculation if either tolerance or iteration condition is met
######################################################################
print('Guess X=\n'+str(X) + '\nsolve A=\n' +str(A) + '  using ' +str(Iterations) + ' Iterations, Abs.Tol=' + str(Tolerance))

def jacobi(A, b, N, x):
                                                                                                                                                                   
    # (1) Create a vector using the diagonal elemts of A
    D = diag(A)
    # (2) Subtract D vector from A into the new vector R
    R = A - diagflat(D)
    # (3) We must Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x

A = array([[2.0,1.0],[5.0,7.0]])
b = array([11.0,13.0])
x = array([1.0,1.0])
sol = jacobi(A,b, 250000, x)

print ('X=' +str(sol) +'\tANS')

Guess X=
[[ 2.9]
 [-3.5]]
solve A=
[[2. 1.]
 [5. 7.]]  using 1000 Iterations, Abs.Tol=0.01

Output:
eigenvalues= [ 2.23606798 -2.23606798]
X=[ 7.11111111 -3.22222222]	ANS


In [24]:
from numpy import array, zeros, diag, diagflat, dot

def Jacobi(A, b, N, x):
                                                                                                                                                                   
    # (1) Create a vector using the diagonal elemts of A
    D = diag(A)
    # (2) Subtract D vector from A into the new vector R
    R = A - diagflat(D)
    print('\nOutput:\neigenvalues= ' + str(np.linalg.eigvals(R)))
    # (3) We must Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x

A = array([[2.0,1.0],[5.0,7.0]])
b = array([11.0,13.0])
x = array([1.0,1.0])
sol = Jacobi(A,b, 250000, x)

print("Solution for x is: ", sol)


Output:
eigenvalues= [ 2.23606798 -2.23606798]
('Solution for x is: ', array([ 7.11111111, -3.22222222]))
